# Iberia01 - A new gridded dataset of daily precipitation and temperatures over Iberia.

Abstract: Iberia01 is a gridded dataset of daily precipitation and mean, maximum and minimum daily temperatures over the Iberian Peninsula covering the period 1971-2015 and reaching a spatial resolution of 0.1º. This dataset has been developed based on previous works done for Spain and Portugal. The present annex includes the code, in R, to reproduce some of the figures shown in the paper Herrera et al. 2019 submitted to Earth System Science Data (ESSD).

## S. Herrera, R.M. Cardoso, P.M.M. Soares, F. Espírito-Santo & J. M Gutiérrez

## R PACKAGE REQUIREMENTS 

The following R packages (R Core Team 2017) are required to follow the examples. Note the last **Session Information Section** of this manual, where the specific package versions used to build these examples are indicated.

### Installing the adequate version

All the *climate4R* packages are currently under active development. Thus, to ensure the reproducibility of these examples, it is recommend to install the same versions of the packages that produced this document, even though in most cases more recent versions exist. The package versions used are indicated in the Session Information Section at the end of this document. The function `install_github` from package `devtools` (Wickham and Chang 2016) is recommended to install a particular package version from the GitHub repositories. For, instance, to install `transformeR` v0.0.14, it suffices with pointing to the specific version tag after the `@` symbol. 

Thus, in order to install the proper versions of the packages of the *climate4R* bundle needed to run these examples: 
  

In [ ]:
## devtools::install_github(c("SantanderMetGroup/transformeR", "SantanderMetGroup/convertR",
                           "SantanderMetGroup/loadeR", "SantanderMetGroup/visualizeR"))

In [ ]:
rm(list = ls())
## options(java.parameters = "-Xmx8000m")
library(loadeR, lib.loc = "~/R/x86_64-pc-linux-gnu-library/3.5/")

### R package for data loading

The R package `loadeR` will perform the data loading task, including authentication against the Santander User Data Gateway (UDG) server (see the [installation instructions](http://meteo.unican.es/ecoms-udg/RPackage/versions)) and enabling the access to any remote dataset via OPeNDAP (more details in the [loadeR's wiki page](https://github.com/SantanderMetGroup/loadeR/wiki)), as well as creating and accesing datasets locally.
                     
Prior to data access through the Santander User Data Gateway, authentication is required to access the UDG. The authentication is performed in one step:

loginUDG(username = "jDoe", password = "*****")

### R package for data post-processing and plotting

In addition, the R package `transformeR` (Bedia and Iturbide 2017) enables climate data transformation (plotting, aggregation, subsetting, PCA/EOF analysis ...). Further details on its capabilities and installation instructions are available in the [wiki page](https://github.com/SantanderMetGroup/transformeR/wiki). It is seamlessly integrated with the data structures provided by `loadeR.ECOMS`.

In [ ]:
library(transformeR, lib.loc = "~/R/x86_64-pc-linux-gnu-library/3.5/")
library(convertR, lib.loc = "~/R/x86_64-pc-linux-gnu-library/3.5/")

### Additional packages

#### Functions for Extreme Value Distributions (evd):

In order to illustrate the analysis included in Herrera et al. 2019 the [evd](https://cran.r-project.org/web/packages/evd/index.html) package should be installed to obtain the Generalized Extreme Value (GEV) distribution.

In [ ]:
## install.packages("evd")
library(evd, lib.loc = "~/R/x86_64-pc-linux-gnu-library/3.5/")

In order to extend the analysis shown in this document to other indices, the reader can consider the following R packages to easily estimate other climate and drought indices.

#### R package for Climate Indices calculation: 

`climate4R.indices` is the package to compute several indices within the climate4R framework, therefore is seamlessly integrated with the climate4R data structures, and provides support for parallel computing.

In [ ]:
## devtools::install_github(c("SantanderMetGroup/climate4R.indices"))
library(climate4R.indices, lib.loc = "~/R/x86_64-pc-linux-gnu-library/3.5/")
indexShow()
?indexGrid   # see the examples 

#### Colour Palettes:

The package `RColorBrewer` is used to replicate the spectral color palette used in the paper in the correlation maps. Next, the palette `veri.colors` is defined, that will be used in the verification maps:

In [ ]:
library(RColorBrewer, lib.loc = "~/R/x86_64-pc-linux-gnu-library/3.5/")
cols <- brewer.pal(n = 11, name = "Spectral")
veri.colors <- colorRampPalette(rev(cols))

## Daily Precipitation Climatology:

First, we obtain the daily precipitation climatology and the differences between both datasets, Iberia01 and E-OBS v17.0. To this aim, the dataset and geographical domain should be defined:

In [ ]:
eobsNcML <- "http://opendap.knmi.nl/knmi/thredds/dodsC/e-obs_0.25regular/rr_0.25deg_reg_v17.0.nc"
ib01NcML <- "http://meteo.unican.es/tds5/dodsC/Iberia01/Iberia01_v1.0_010reg_aa_3d.ncml"
## Defining the temporal and geographical domain:
latLim <- c(34, 44)
lonLim <- c(-10, 6)
years <- c(1971:2010)

### E-OBS v17.0:

First, we obtain the climatology for the E-OBS v17.0 dataset. To this aim we should know the variables included in the dataset

In [ ]:
di <- dataInventory(eobsNcML)
## Variables:
names(di)

and load the corresponding data:

In [ ]:
pr.eobs <- loadGridData(eobsNcML, "rr", dictionary = FALSE, lonLim = lonLim, latLim = latLim, 
                           season = c(1:12), years = years, time = "DD", aggr.d = "sum")
setGridUnits(pr.eobs, "mm", var = "rr")
pr.eobs <- climatology(pr.eobs, clim.fun = list(FUN = "mean", na.rm = TRUE))
spatialPlot(pr.eobs, backdrop.theme = "countries", at = seq(-0.5,6,0.5), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Precipitation (1971-2010)")

### Iberia01:

Now, we consider the Iberia01 dataset:

In [ ]:
di <- dataInventory(ib01NcML)
## Variables:
names(di)

In [ ]:
pr.Ib01 <- loadGridData(ib01NcML, "pr", dictionary = FALSE, lonLim = lonLim, latLim = latLim, 
                           season = c(1:12), years = years, time = "DD", aggr.d = "sum")
setGridUnits(pr.Ib01, "mm", var = "pr")
pr.Ib01 <- climatology(pr.Ib01, clim.fun = list(FUN = "mean", na.rm = TRUE))
spatialPlot(pr.Ib01, backdrop.theme = "countries", at = seq(-0.5,6,0.5), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Precipitation (1971-2010)")

In order to obtain the difference between both datasets, we interpolate E-OBS v17.0 to the grid of the Iberian01 dataset:

In [ ]:
## E-OBS climatology is regridded to the Iberian01 grid:
pr.eobs <- interpGrid(pr.eobs, new.coordinates = getGrid(pr.Ib01), method = "nearest")

### Differences between E-OBS v17.0 and Iberia01:
- Bias:

In [ ]:
bias <- gridArithmetics(pr.eobs, pr.Ib01, operator = "-")
spatialPlot(bias, backdrop.theme = "countries", at = seq(-3,3,0.5), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Bias (1971-2010)")

- Relative Bias:

In [ ]:
## Relative Bias
C <- 100
relBias <- gridArithmetics(pr.eobs, pr.Ib01, pr.Ib01, C, operator = c("-", "/", "*"))
spatialPlot(relBias, backdrop.theme = "countries", at = seq(-100,100,10), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Relative Bias (1971-2010)")

In [ ]:
## Cleaning the workspace:
rm(list=c("pr.Ib01", "pr.eobs", "bias", "relBias"))

## Mean Daily Temperature:

Once the climatologies for some precipitation indices have been obtained we show the climatology of mean temperature. Note that, in this case, the datasets should be redefined, at least for E-OBS v17.0:

In [ ]:
eobsNcML <- "http://opendap.knmi.nl/knmi/thredds/dodsC/e-obs_0.25regular/tg_0.25deg_reg_v17.0.nc"
ib01NcML <- "http://meteo.unican.es/tds5/dodsC/Iberia01/Iberia01_v1.0_010reg_aa_3d.ncml"
## Defining the temporal and geographical domain:
latLim <- c(34, 44)
lonLim <- c(-10, 6)
years <- c(1971:2010)

### E-OBS v17.0:

First, we obtain the climatology for the E-OBS v17.0 dataset. To this aim we should know the variables included in the dataset

In [ ]:
di <- dataInventory(eobsNcML)
## Variables:
names(di)

and load the corresponding data:

In [ ]:
tg.eobs <- loadGridData(eobsNcML, "tg", dictionary = FALSE, lonLim = lonLim, latLim = latLim, 
                           season = c(1:12), years = years, time = "DD", aggr.d = "mean")
tg.eobs <- climatology(tg.eobs, clim.fun = list(FUN = "mean", na.rm = TRUE))
spatialPlot(tg.eobs, backdrop.theme = "countries", at = seq(0,25,1), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Mean Temperature (1971-2010)")

### Iberia01:

Now, we consider the Iberia01 dataset:

In [ ]:
di <- dataInventory(ib01NcML)
## Variables:
names(di)

In [ ]:
tg.Ib01 <- loadGridData(ib01NcML, "tas", dictionary = FALSE, lonLim = lonLim, latLim = latLim, 
                           season = c(1:12), years = years, time = "DD", aggr.d = "mean")
tg.Ib01 <- climatology(tg.Ib01, clim.fun = list(FUN = "mean", na.rm = TRUE))
spatialPlot(tg.Ib01, backdrop.theme = "countries", at = seq(0,25,1), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Mean Temperature (1971-2010)")

In order to obtain the difference between both datasets, we interpolate E-OBS v17.0 to the grid of the Iberian01 dataset:

In [ ]:
## E-OBS climatology is regridded to the Iberian01 grid:
tg.eobs <- interpGrid(tg.eobs, new.coordinates = getGrid(tg.Ib01), method = "nearest")

### Differences between E-OBS v17.0 and Iberia01:
- Bias:

In [ ]:
bias <- gridArithmetics(tg.eobs, tg.Ib01, operator = "-")
spatialPlot(bias, backdrop.theme = "countries", at = seq(-5,5,0.5), 
            col.regions = veri.colors(51), colorkey = list(space = "bottom"), 
            main = "Bias (1971-2010)")

In [ ]:
## Cleaning the workspace:
rm(list=c("tg.Ib01", "tg.eobs", "bias"))

## References:

* Bedia, J. and Iturbide, M., 2017. `transformeR`: Climate data post-processing. R package version 0.0.9. https://github.com/SantanderMetGroup/transformeR/wiki.
* Frías M.D., Iturbide M., Manzanas R., Bedia J., Fernández J., Herrera S., Cofiño A.S., Gutiérrez J.M., 2018. An R package to visualize and communicate uncertainty in seasonal climate prediction, Environmental Modelling & Software, 99, 101-110, https://doi.org/10.1016/j.envsoft.2017.09.008.
* Gutiérrez J. M., Bedia J., Iturbide M., Herrera S., Manzanas R., Baño-Medina J., Frías M. D., San-Martín D., Fernández J., Cofiño A. S., 2018. Climate Research Reproducibility with the Climate4R R-based framework; 8 th International Workshop on Climate Information. Available on-line here: http://www.meteo.unican.es/work/climate4r/climate4r_CI2018.pdf
* R Core Team, 2017. R: A language and environment for statistical computing. R Foundation for Statistical Computing, Vienna, Austria. URL https://www.R-project.org/.
* Wickham, H., Hester, J. and Chang, W., 2018. `devtools`: Tools to Make Developing R Packages Easier. R package version 2.0.1. https://CRAN.R-project.org/package=devtools

## Session Information and Package Versions

In [ ]:
print(sessionInfo(package = c("loadeR", "convertR", "transformeR",
                        "visualizeR", "RColorBrewer")))